# Web-scraping-utility for token prices
*https://coinmarketcap.com/all/views/all/*

First of all, import all the packages needed for the website

In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
from pandas import ExcelWriter
from tqdm import tqdm
import datetime

Create two empty lists for storing all the tokens addresses and ID's.

In [2]:
links = []
tokens = []

Then, look for all the token addresses within the main webpage and append them to the links list.

In [3]:
%%time
main_webpage = 'https://coinmarketcap.com/all/views/all/'
sauce = urllib.request.urlopen(main_webpage).read()
soup = bs.BeautifulSoup(sauce, 'lxml')

table = soup.find('table')
for url in table.find_all('a'):
	page_address = url.get('href')
	first_char = page_address[:1]
	last_char = page_address[-1:]
	if (first_char == '/') and (last_char == '/'):
		links.append(page_address)

Wall time: 2.84 s


Drop the duplicates links in the list.

In [4]:
links = list(set(links))
ln = len(links)

Get the tokens ID's.

In [5]:
for row in links:
	row = row.replace('/currencies/', '')
	row = row.replace('/', '')
	tokens.append(row)

Now create a new empty dataframe where all the values will be stored.

In [6]:
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap', 'Token']
data = pd.DataFrame(columns = columns)

Loop and extract the information for each token.

In [7]:
%%time
for address, tkn in tqdm(zip(links, tokens), total = ln):   
	webpage = 'https://coinmarketcap.com' + address + 'historical-data/?start=20130428&end=20180103'
	sauce = urllib.request.urlopen(webpage).read()
	soup = bs.BeautifulSoup(sauce, 'lxml')
	df = pd.read_html(webpage)[0]
	df['Token'] = tkn
    
	try:
		data = data.append(df, ignore_index=True)
	except:
		pass

100%|██████████████████████████████████████| 1385/1385 [40:07<00:00,  2.17s/it]


Wall time: 40min 7s


Clean the data:
1. Drop the empty rows
2. Replace the '-' with zeros
3. Change the datatype
4. Fix the Date column

In [8]:
data = data[data.Date != 'No data was found for the selected time period.']
data['Volume'] = data['Volume'].replace('-', 0)
data['Market Cap'] = data['Market Cap'].replace('-', 0)
data['Volume'] = data['Volume'].astype('int64')
data['Market Cap'] = data['Market Cap'].astype('int64')

Use the dictionary to map the months.

In [9]:
dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
		'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
		'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

Extract the dates and fix the Date column.

In [10]:
day = data.Date.str.slice(4, 6)
month = data.Date.str.slice(0, 3)
month = month.map(dict)
year = data.Date.str.slice(8, 12)
s = year + '/' + month + '/' + day
data.Date = pd.to_datetime(s)

Export to Excel

In [11]:
%%time
name = 'Coinmarketcap_data.xlsx'
writer = pd.ExcelWriter(name)		
data.to_excel(writer, index = False)
writer.save()

Wall time: 2min 37s
